In [ ]:
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
import pandas as pd

from harp_resources import process, utils
from scipy.signal import savgol_filter
import matplotlib.patches as patches

In [ ]:
data_path = Path('/home/ikharitonov/RANCZLAB-NAS/data/ONIX/20240730_Mismatch_Experiment/GRAB_MMclosed&Regular_220824/2024-08-22T13-13-15_B3M6')
photometry_path = Path('/home/ikharitonov/RANCZLAB-NAS/data/ONIX/20240730_Mismatch_Experiment/GRAB_MMclosed&Regular_220824/photometry/B3M6_MMclosed&Regular_day1/2024_08_22-15_16_40')

In [ ]:
def moving_average_smoothing(X,k):
    S = np.zeros(X.shape[0])
    for t in range(X.shape[0]):
        if t < k:
            S[t] = np.mean(X[:t+1])
        else:
            S[t] = np.sum(X[t-k:t])/k
    return S

def running_unit_conversion(running_array):
    resolution = 12000 # counts per inch
    inches_per_count = 1 / resolution
    meters_per_count = 0.0254 * inches_per_count
    dt = 0.01 # for OpticalTrackingRead0Y(46)
    linear_velocity = meters_per_count / dt # meters per second per count
    
    # ball_radius = 0.1 # meters 
    # angular_velocity = linear_velocity / ball_radius # radians per second per count
    # angular_velocity = angular_velocity * (180 / np.pi) # degrees per second per count
    # print(angular_velocity)
    
    return running_array * linear_velocity * 100

In [ ]:
preprocessed_csv = pd.read_csv('/home/ikharitonov/Downloads/preprocessed_grab.csv')
preprocessed_csv['TimeStamp'] = preprocessed_csv['TimeStamp'] * 1000
preprocessed_csv

In [ ]:
streams = utils.load_registers(data_path)

In [ ]:
conversions = process.calculate_conversions_second_approach(data_path, photometry_path, verbose=False)

In [ ]:
conversions.keys()

In [ ]:
preprocessed_csv['HARP Timestamps'] = conversions['photometry_to_harp_time'](preprocessed_csv['TimeStamp'])
preprocessed_csv['HARP Seconds'] = process.convert_datetime_to_seconds(preprocessed_csv['HARP Timestamps'])
preprocessed_csv

In [ ]:
OnixAnalogClock = utils.read_OnixAnalogClock(data_path)
OnixAnalogData = utils.read_OnixAnalogData(data_path, binarise=True)

In [ ]:
print('Photodiode start', conversions['onix_to_harp_timestamp'](OnixAnalogClock[0]))
print('Photodiode stop', conversions['onix_to_harp_timestamp'](OnixAnalogClock[-1]))
print('Photometry start', preprocessed_csv['HARP Timestamps'].iloc[0])
print('Photometry stop', preprocessed_csv['HARP Timestamps'].iloc[-1])

In [ ]:
ExperimentEvents = utils.read_ExperimentEvents(data_path)
ExperimentEvents

In [ ]:
ExperimentEvents.Value.unique()

In [ ]:
ExperimentEvents[ExperimentEvents.Value=='Apply halt: 1s']

In [ ]:
ExperimentEvents[ExperimentEvents.Value=='LinearMismatch block started']

In [ ]:
# A = ExperimentEvents[ExperimentEvents.Value=='Apply halt: 1s'].iloc[0].Seconds
# B = ExperimentEvents[ExperimentEvents.Value=='Apply halt: 1s'].iloc[-1].Seconds

# A = ExperimentEvents.iloc[0].Seconds
# B = ExperimentEvents.iloc[-1].Seconds

A = ExperimentEvents[ExperimentEvents.Value=='LinearMismatch block started'].iloc[0].Seconds
B = ExperimentEvents.iloc[-1].Seconds

# A = 354900
# B = A+200

print(A, B)
print(process.convert_seconds_to_datetime(A), process.convert_seconds_to_datetime(B))

In [ ]:
selected_harp_times, selected_photodiode_data = process.select_from_photodiode_data(OnixAnalogClock, OnixAnalogData, A, B, conversions)

In [ ]:
# fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(12,12))

plt.figure(figsize=(12,6))
plt.plot(preprocessed_csv[preprocessed_csv['HARP Seconds'].between(A, B)]['HARP Seconds'], preprocessed_csv[preprocessed_csv['HARP Seconds'].between(A, B)]['470_dfF'], label='470nm')

t = (selected_harp_times - utils.harp.REFERENCE_EPOCH).total_seconds()
plt.plot(t, selected_photodiode_data[:,0], label='Photodiode')
plt.xlabel('HARP Seconds')
plt.ylabel('df/F (%)')
plt.legend()
plt.show()

plt.figure(figsize=(12,6))
# running in cm/s
y1 = streams['H1']['OpticalTrackingRead0X(46)'].loc[process.convert_seconds_to_datetime(A):process.convert_seconds_to_datetime(B)]
y1 = running_unit_conversion(y1)
t = (y1.index - utils.harp.REFERENCE_EPOCH).total_seconds()
# y2 = savgol_filter(y1, 50, 3)
y3 = moving_average_smoothing(y1, 50)
# plt.plot(t, y1, label='Raw running')
# plt.plot(t, y2, label='Savgol')
plt.plot(t, y3, label='Moving average')
plt.legend()
plt.xlabel('HARP Seconds')
plt.ylabel('running (cm/s)')
plt.show()

## halt time analysis

In [ ]:
t = (selected_harp_times - utils.harp.REFERENCE_EPOCH).total_seconds()
t

In [ ]:
photodiode_low_state_times = t[np.where(selected_photodiode_data[:,0]==0)].to_numpy()
intervals_between_states = np.diff(photodiode_low_state_times)
print(photodiode_low_state_times)
print(intervals_between_states)
print(photodiode_low_state_times.shape, intervals_between_states.shape)

In [ ]:
# Checking what are the differences between the time values that we selected are (those corresponding to low states of photodiode)
counts, intervals, _ = plt.hist(intervals_between_states, bins=100)
print(counts)
print(intervals)

In [ ]:
# low many values there are in the smallest interval
counts[0]

In [ ]:
# mean of differences plus two standard deviations
threshold = intervals_between_states.mean() + 1 * intervals_between_states.std()
threshold

In [ ]:
intervals_between_states[np.where(intervals_between_states < threshold)]

In [ ]:
print(f'The number of values in the first interval [{intervals[0]}, {intervals[1]}] = {counts[0]} == the number of difference values between halt occurrences < chosen threshold {threshold} = {intervals_between_states[np.where(intervals_between_states < threshold)].shape[0]} ==> {counts[0]==intervals_between_states[np.where(intervals_between_states < threshold)].shape[0]}')

In [ ]:
intervals_between_states[np.where(intervals_between_states >= threshold)]

In [ ]:
# Selecting the indices where there are large time gaps between low states of photodiode (which are disconnected - distinct halts)
inds = np.where(intervals_between_states >= threshold)[0] + 1
inds

In [ ]:
# Getting the HARP second values for these halt beginning events
halt_times = photodiode_low_state_times[inds]
halt_times

In [ ]:
# Doing a visual check the detected halt time correspond to the photodiode trace
plt.figure(figsize=(12,6))
plt.plot(preprocessed_csv[preprocessed_csv['HARP Seconds'].between(A, B)]['HARP Seconds'], preprocessed_csv[preprocessed_csv['HARP Seconds'].between(A, B)]['470_dfF'])

t = (selected_harp_times - utils.harp.REFERENCE_EPOCH).total_seconds()
plt.plot(t, selected_photodiode_data[:,0])

for halt_time in halt_times:
    plt.axvline(halt_time, c='r', alpha=0.5)

plt.show()

## block average F0 analysis

In [ ]:
def select_segment(trace, time_column_name, start, end):
    return trace[trace[time_column_name].between(start, end)]

def get_perievent_trace(trace, time_column_name, event_time, before_event_period=5, during_event_period=1, after_event_period=5):
    pre_event_time = event_time - before_event_period
    after_event_time = event_time + during_event_period + after_event_period
    return select_segment(trace, time_column_name, pre_event_time, after_event_time)

def dfF(trace, F0):
    return (trace - F0) / F0

In [ ]:
ExperimentEvents.Value.unique()

In [ ]:
temp_df = ExperimentEvents[ExperimentEvents.Value.isin(['LinearNormal block started', 'LinearRegularMismatch block started', 'LinearMismatch block started'])]
temp_df

In [ ]:
block1_start = preprocessed_csv['HARP Seconds'].iloc[0]
block2_start = temp_df.iloc[1]['Seconds']
block3_start = temp_df.iloc[2]['Seconds']
block3_end = preprocessed_csv['HARP Seconds'].iloc[-1]

print(block1_start, block2_start, block3_start, block3_end)

In [ ]:
temp_data = preprocessed_csv[['HARP Seconds','470_dfF']]
block1_average_dfF = select_segment(temp_data, 'HARP Seconds', block1_start, block2_start)['470_dfF'].mean()
block1_std_dfF = select_segment(temp_data, 'HARP Seconds', block1_start, block2_start)['470_dfF'].std()
block2_average_dfF = select_segment(temp_data, 'HARP Seconds', block2_start, block3_start)['470_dfF'].mean()
block2_std_dfF = select_segment(temp_data, 'HARP Seconds', block2_start, block3_start)['470_dfF'].std()
block3_average_dfF = select_segment(temp_data, 'HARP Seconds', block3_start, block3_end)['470_dfF'].mean()
block3_std_dfF = select_segment(temp_data, 'HARP Seconds', block3_start, block3_end)['470_dfF'].std()

print('block1 mean', block1_average_dfF, 'std', block1_std_dfF)
print('block2 mean', block2_average_dfF, 'std', block2_std_dfF)
print('block3 mean', block3_average_dfF, 'std', block3_std_dfF)

In [ ]:
plt.figure(figsize=(5,3))

plt.scatter(0,block1_average_dfF, c='black')
# plt.errorbar(0, block1_average_dfF, yerr=block1_std_dfF, capsize=5, c='black')

plt.scatter(1,block2_average_dfF, c='black')
# plt.errorbar(1, block2_average_dfF, yerr=block2_std_dfF, capsize=5, c='black')

plt.scatter(2,block3_average_dfF, c='black')
# plt.errorbar(2, block3_average_dfF, yerr=block3_std_dfF, capsize=5, c='black')

plt.xticks([0,1,2], ['Baseline', 'Regular mismatches', 'Random mismatches'])
plt.ylabel('average df/F (%)')

plt.ylim([-0.2, 0.2])

plt.show()

## perievent analysis

In [ ]:
temp_data = preprocessed_csv[preprocessed_csv['HARP Seconds'].between(A, B)][['HARP Seconds','470_dfF']]
get_perievent_trace(temp_data, 'HARP Seconds', halt_times[0])

In [ ]:
running_unit_conversion(streams['H1']['OpticalTrackingRead0X(46)'])

In [ ]:
moving_average_smoothing(running_unit_conversion(streams['H1']['OpticalTrackingRead0X(46)']), 100).shape

In [ ]:
running_unit_conversion(streams['H1']['OpticalTrackingRead0X(46)']).rolling(5).sum()

In [ ]:
# all_running = moving_average_smoothing(running_unit_conversion(streams['H1']['OpticalTrackingRead0X(46)']), 100)
# all_running = running_unit_conversion(streams['H1']['OpticalTrackingRead0X(46)']).apply(lambda x: moving_average_smoothing(x, 100))
all_running = running_unit_conversion(streams['H1']['OpticalTrackingRead0X(46)'])

In [ ]:
before_event_period=5
during_event_period=1
after_event_period=5

photometry_chunks = []
running_chunks = []

for halt_time in halt_times:
    
    # Select chunk
    photometry_chunk = get_perievent_trace(temp_data, 'HARP Seconds', halt_time)
    start = process.convert_seconds_to_datetime(halt_time - before_event_period)
    end = process.convert_seconds_to_datetime(halt_time + during_event_period + after_event_period)
    running_chunk = all_running.loc[start:end]
    running_times = (running_chunk.index - utils.harp.REFERENCE_EPOCH).total_seconds()
    
    start = halt_time - before_event_period
#     start = halt_time - 1
    end = halt_time
    photometry_F0 = select_segment(photometry_chunk, 'HARP Seconds', start, end)['470_dfF'].mean()
    running_F0 = running_chunk.loc[process.convert_seconds_to_datetime(start):process.convert_seconds_to_datetime(end)].mean()
    
    photometry_chunk = dfF(photometry_chunk, photometry_F0)
    running_chunk = dfF(running_chunk, running_F0)
    
    photometry_chunks.append(photometry_chunk['470_dfF'].values)
    running_chunks.append(running_chunk.values)

photometry_chunks = np.array(photometry_chunks)
running_chunks = np.array(running_chunks)
print(photometry_chunks.shape, running_chunks.shape)

average_photometry_chunk = photometry_chunks.mean(axis=0)
average_running_chunk = running_chunks.mean(axis=0)

In [ ]:
photometry_t = np.linspace(-before_event_period, during_event_period + after_event_period, average_photometry_chunk.shape[0])
running_t = np.linspace(-before_event_period, during_event_period + after_event_period, average_running_chunk.shape[0])

plt.plot(photometry_t, average_photometry_chunk, label='GRAB df/F (%)')
plt.plot(running_t, average_running_chunk, c='black', label='running')
# plt.axvline(0, c='r', alpha=0.5)

plt.gca().add_patch(patches.Rectangle((0, plt.gca().get_ylim()[0]), 1, plt.gca().get_ylim()[1]-plt.gca().get_ylim()[0], edgecolor='none',facecolor='red', alpha=0.5))

plt.xlabel('time from halt (s)')
plt.ylabel('relative change (%)')

plt.legend()

plt.show()